# Feedback Analysis Example
## Prerequisites
Download conversation log using Athena. Following is an example query to get conversation with user's feedback:

```sql
SELECT 
    d.newimage.PK.S AS UserId,
    d.newimage.SK.S AS ConvId,
    d.newimage.TotalPrice.N AS TotalPrice,
    d.newimage.MessageMap.S AS MessageMap
FROM 
    bedrockchatstack_usage_analysis.ddb_export d
WHERE 
    d.datehour BETWEEN '2024/04/01/00' AND '2024/04/29/23'
    AND REGEXP_LIKE(d.newimage.MessageMap.S, '"feedback":((?!\snull).)')
ORDER BY
    d.datehour DESC
```

In [ ]:
import pandas as pd
import json

In [ ]:
from IPython import display

def extract_conversation(data):
    # Extract conversation to display.
    conversation = []

    for key, value in data.items():
        if key not in ("system", "instruction"):
            # print(f"key: {key}")
            # print(f"value: {value}")
            role = value['role']
            feedback = value.get('feedback')
            used_chunks = value.get('used_chunks')
            content = value['content'][0]['body']

            content = content.replace('\\n', '\n')

            conversation.append(f"### {role.capitalize()}:\n\n{content.strip()}")
            if feedback:
                category = feedback.get('category')
                comment = feedback.get('comment')
                conversation.append(f"### feedback:\n\ncategory: {category}\n\ncomment: {comment}")
                
            if used_chunks:
                conversation.append(f"### used_chunks:\n\n{used_chunks}")

    return "\n\n".join(conversation)

In [ ]:
# Enter downloaded csv file path from athena
filepath = './xxx.csv'
df = pd.read_csv(filepath)
df.head()

In [ ]:
# Display conversation with feedback and used RAG chunks
display.Markdown(extract_conversation(json.loads(df["MessageMap"].iloc[1])))